In [18]:
#HW-3 : Mohammed Farees Patel

import re
import csv
import string
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')
from sklearn.feature_extraction.text import TfidfVectorizer

In [50]:
data = pd.read_csv('Reviews.csv',encoding='utf8')

In [22]:
def cleaning(review, remove_stopwords=True):
   
    review_text = re.sub("[^a-zA-Z]"," ", review)   
    
    words = review_text.lower().split()
    
    b=[]
    stemmer = english_stemmer 
    for word in words:
        b.append(stemmer.stem(word))
    
    return(b)

In [23]:
clean_Text_sw = []
for review in data['Text']:
    clean_Text_sw.append( " ".join(cleaning(review)))

In [24]:
word_list = [word for line in clean_Text_sw for word in line.split()]

In [25]:
#Part (a) : Identify all the unique words that appear in the "review/text" field of the reviews. Denote the set of such words as L.
L=list(set(word_list))
L

['rondnoir',
 'frankencorn',
 'acidophuilus',
 'sti',
 'softish',
 'rzwiy',
 'brack',
 'collag',
 'cherryland',
 'jbllk',
 'outther',
 'minichur',
 'kclub',
 'sugarari',
 'land',
 'overrun',
 'srrength',
 'leviti',
 'compiback',
 'reappli',
 'smyrna',
 'impecc',
 'antioxyd',
 'flouro',
 'iuv',
 'weezi',
 'pregrind',
 'asjust',
 'cookvill',
 'ktja',
 'rhodiola',
 'fracisco',
 'nuernberg',
 'willard',
 'wayyyyyyy',
 'creel',
 'punicalagin',
 'overexercis',
 'dogloo',
 'peepster',
 'raita',
 'reyna',
 'sunett',
 'zhat',
 'bachatero',
 'flagpol',
 'basic',
 'oughterard',
 'prorat',
 'arrov',
 'produxt',
 'unders',
 'sobliro',
 'davjohn',
 'retangular',
 'tcha',
 'snx',
 'rawhideish',
 'mimiccrem',
 'sacha',
 'pawi',
 'putuo',
 'wegemann',
 'slept',
 'subcontin',
 'sqeak',
 'lonesom',
 'ftc',
 'protrus',
 'kink',
 'parika',
 'brailsford',
 'mcvitt',
 'spoonabl',
 'sub',
 'jsz',
 'andalso',
 'disentangl',
 'societi',
 'aplic',
 'faster',
 'womb',
 'eeeek',
 'interoper',
 'dashida',
 'oiwu',


In [26]:
#Part (b) : Loading stopword file
global stopword_list
f_open = open('stopwords.txt','r')
sw_list = f_open.read()
f_open.close()
stopword_list = sw_list

In [32]:
def remove_stopwords( review, remove_stopwords=True):
   
    

    review_text = re.sub("[^a-zA-Z]"," ", review)
   
    words = review_text.lower().split()
    
    if remove_stopwords:
        words = [w for w in words if not w in stopword_list]

    b=[]
    stemmer = english_stemmer 
    for word in words:
        b.append(stemmer.stem(word))

    
    return(b)

In [33]:
clean_Text = []
for review in data["Text"]:
    clean_Text.append( " ".join(remove_stopwords(review)))

In [29]:
word_list = [word for line in clean_Text for word in line.split()]

In [34]:
#Part (b) : Remove from L all stopwords in "Long Stopword List" from http://www.ranks.nl/stopwords. Denote the cleaned set as W.
W=word_list

In [35]:
#Part (c) : Count the number of times each word in W appears among all reviews ("review/text" field) and identify the top 500 words.
Top_Words_Review =pd.Series(' '.join(clean_Text).lower().split()).value_counts()[:500]
print ("Top Count Words Used In Review", Top_Words_Review)

Top Count Words Used In Review tast           265641
flavor         222049
good           205356
product        201678
love           188117
coffe          176338
great          168587
food           153393
dog            107706
eat            106855
amazon         106639
buy            106587
order           92513
cup             90432
bag             89009
price           87673
well            80925
find            79859
drink           78750
best            76870
better          70775
store           69517
treat           68892
day             68477
box             67574
water           65027
cat             64919
chocol          64497
recommend       61540
sweet           61187
                ...  
refresh          6790
seller           6773
expir            6761
crave            6757
microwav         6749
stash            6735
beat             6724
tomato           6672
last             6669
loos             6665
oat              6636
effect           6628
sour             6572
d

In [82]:
#Part (c) : Save the top 500 words in a csv file for submission
Top_Words_Review.to_excel("500_top_words.xlsx")

In [36]:
#Part (d) : Vectorize all reviews ("review/text" field) using these 500 words.
vectorizer = TfidfVectorizer(min_df=4, max_features = 500)
review_vectorizer = vectorizer.fit_transform(clean_Text)

In [38]:
#Part (e) : Cluster the vectorized reviews into 10 clusters using k-means.

from sklearn.cluster import MiniBatchKMeans

num_clusters = 10
kmeans_model = MiniBatchKMeans(n_clusters=num_clusters, init='k-means++', n_init=1, 
                         init_size=1000, batch_size=1000, verbose=False, max_iter=1000)
kmeans = kmeans_model.fit(review_vectorizer)
kmeans_clusters = kmeans.predict(review_vectorizer)
kmeans_distances = kmeans.transform(review_vectorizer)

In [43]:
#Part (f) : From each centroid, select the top 5 words that represent the centroid
sorted_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(num_clusters):
    print("Cluster %d:" % (i+1))
    for j in sorted_centroids[i, :5]:
        print(' %s: %d' % (terms[j],j))
    print()

Cluster 1:
 good: 206
 tast: 450
 flavor: 190
 price: 353
 great: 211

Cluster 2:
 cat: 68
 food: 192
 eat: 154
 love: 273
 treat: 469

Cluster 3:
 sauc: 389
 oil: 310
 cook: 107
 salt: 385
 popcorn: 344

Cluster 4:
 coffe: 93
 cup: 121
 flavor: 190
 tast: 450
 roast: 380

Cluster 5:
 product: 357
 great: 211
 tast: 450
 love: 273
 good: 206

Cluster 6:
 water: 484
 tast: 450
 drink: 148
 peanut: 326
 butter: 56

Cluster 7:
 tast: 450
 flavor: 190
 love: 273
 great: 211
 drink: 148

Cluster 8:
 chocol: 83
 chip: 82
 bar: 24
 cooki: 108
 snack: 413

Cluster 9:
 amazon: 12
 order: 317
 store: 435
 price: 353
 find: 185

Cluster 10:
 dog: 145
 treat: 469
 food: 192
 love: 273
 chew: 78



In [48]:
f = open('Cluster_feature_value.txt','w')
for i in range(num_clusters):
    f.write("Cluster %d:" % (i+1))
    for j in sorted_centroids[i, :5]:
        f.write("\n")
        f.write(' %s: %d' % (terms[j],j))
    f.write("\n\n")
f.close()